In [1]:
#clear all
%reset -f

#import packages
import numpy as np
import scipy
import sys
import os
import pandas as pd
import mne
import matplotlib
import h5py
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'frn'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [2]:
#setting up path/ directory
#access trial type epoched data for each participant
def load_tfr_epochs(pp_num, root_dir, erp_path, task):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    filename = os.path.join(pp_directory, 'p%03d_%s-epo.fif' % (pp_num, task))

    epochs = mne.read_epochs(filename)

    return epochs, pp_directory

In [3]:
#set up parameters for time-frequency morlet convolution
def set_tfr_params(l_freq = 6, h_freq = 35, num_freq = 50, cycle = 6, samp_rate = 200):
    freqs = np.logspace(*np.log10([l_freq, h_freq]), num = num_freq)
    n_cycles = cycle
    sfreq = samp_rate
    
    return freqs, n_cycles, sfreq

In [4]:
#save tfr'd data
def save_tfr_data(pp_num, root_dir, erp_path, data, task, output):
    # Save the tfr'd data
    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    out_fname = os.path.join(pp_directory, 'p%03d_%s_%s-tfr.h5' % (pp_num, task, output))
    mne.time_frequency.write_tfrs(out_fname, tfr=data, overwrite = True)

In [5]:
#load tfr'd data
def load_tfr_data(pp_num, root_dir, erp_path, task, output):
    # Save the tfr'd data
    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    out_fname = os.path.join(pp_directory, 'p%03d_%s_%s-tfr.h5' % (pp_num, task, output))
    dat = mne.time_frequency.read_tfrs(out_fname)
    
    return dat

In [6]:
# After transforming the TFR data into a single signal representative of frequency and channels
# we compare the two signals: same as taking their difference (signal1 minus signal2) and using this in permutation test
def get_clust_perm_test(conditionA, conditionB, pval, n_permutations):
    #take difference of two conditions
    data = np.subtract(conditionA, conditionB)
    np.shape(data)
    #define cluster forming threshold based on p-value
    df = len(participants) - 1
    thresh = scipy.stats.t.ppf(1 - pval / 2, df)
    #run cluster-based permutation test
    T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh, 
                                                          n_permutations = n_permutations, tail = 0, 
                                                          adjacency = None, seed = 999, 
                                                          out_type = 'mask', verbose = True)

    return T_0, clust_idx, clust_pvals, H0

In [7]:
#save average tfr per participant for each condition
#morlet wavelet convolution for each participant, according to trial types - POWER

# perturb_conds = ['early_late_aligned', 'early_rot', 'late_rot', 'early_mir', 'late_mir', 'early_rdm', 'late_rdm']

# for pcond in range(0, len(perturb_conds)):
#     for pp in participants:
#         data, ppdir = load_tfr_epochs(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond])
#         freqs, n_cycles, sfreq = set_tfr_params()
        
#         #average epochs within participant
#         power = mne.time_frequency.tfr_morlet(
#             data,
#             freqs=freqs,
#             n_cycles=n_cycles,
#             use_fft=False,
#             decim=1,
#             n_jobs=None,
#             return_itc = False,
#             average = True
#         )
        
#         save_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, data = power, task = perturb_conds[pcond], output = "power")

In [8]:
# load in tfr object, ensure to baseline correct; (tfr shape is channels, freqs, timepts)
# narrow down to channels 
# narrow down to frequencies and timepts
# take mean of alpha or beta ranges (freqs)
# take mean across electrodes (channels)

perturb_conds = ['early_late_aligned', 'early_rot', 'late_rot', 'early_mir', 'late_mir', 'early_rdm', 'late_rdm']


# channels = ['Fpz', 'AFz', #frontal (cognitive)
#             'Fz', 'F3', 'F4', 'FCz', #medial frontal (motor preparatory)
#             'Cz', 'C3', 'C4', #motor execution
#             'Pz', 'P3', 'P4', #parietal areas
#             'Oz', 'O1', 'O2'] #visual areas

# FRONTAL CHANNELS
channels = ['Fpz', 'AFz', 'Fz', 'F3', 'F4', 'FCz']
# BASELINE FOR FEEDBACK ONSET
baseline_t = (-1.5, -1.2)

# ALPHA
freq_lower = 8
freq_upper = 13

alpha_frontal_EarlyLateAligned = []
alpha_frontal_EarlyRot = []
alpha_frontal_LateRot = []
alpha_frontal_EarlyMir = []
alpha_frontal_LateMir = []
alpha_frontal_EarlyRdm = []
alpha_frontal_LateRdm = []

for pcond in range(0, len(perturb_conds)):
    allppdat = []
    for pp in participants:
        dat = load_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond], output = "power")
        # apply baseline correction
        dat = dat[0].apply_baseline(baseline_t, mode='logratio')
        # narrow down to channels
        dat = dat.pick_channels(channels)
        #transform to ndarray (channels, freqs, times)
        npdat = dat.data

        # get needed frequency indices
        nfreqs = [] #get indices of frequencies we want
        for i in range(0, len(dat.freqs)):
            if dat.freqs[i] >= freq_lower and dat.freqs[i] <= freq_upper:
                nfreqs.append(i)
        npdat = npdat[:, nfreqs, :]

        # get needed timept indices
        ntimes = list(range(350,701))
        npdat = npdat[:,:,ntimes]
        
        ppdat = np.mean(npdat, axis = 1) #calculate mean across frequencies
        ppdat = np.mean(ppdat, axis = 0) # calculate mean across channels
        
        allppdat.append(ppdat)
        
    if pcond == 0:
        alpha_frontal_EarlyLateAligned.append(allppdat)
    elif pcond == 1:
        alpha_frontal_EarlyRot.append(allppdat)
    elif pcond == 2:
        alpha_frontal_LateRot.append(allppdat)
    elif pcond == 3:
        alpha_frontal_EarlyMir.append(allppdat)
    elif pcond == 4:
        alpha_frontal_LateMir.append(allppdat)
    elif pcond == 5:
        alpha_frontal_EarlyRdm.append(allppdat)
    elif pcond == 6:
        alpha_frontal_LateRdm.append(allppdat)

# BETA
freq_lower = 13
freq_upper = 35

beta_frontal_EarlyLateAligned = []
beta_frontal_EarlyRot = []
beta_frontal_LateRot = []
beta_frontal_EarlyMir = []
beta_frontal_LateMir = []
beta_frontal_EarlyRdm = []
beta_frontal_LateRdm = []

for pcond in range(0, len(perturb_conds)):
    allppdat = []
    for pp in participants:
        dat = load_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond], output = "power")
        # apply baseline correction
        dat = dat[0].apply_baseline(baseline_t, mode='logratio')
        # narrow down to channels
        dat = dat.pick_channels(channels)
        #transform to ndarray (channels, freqs, times)
        npdat = dat.data

        # get needed frequency indices
        nfreqs = [] #get indices of frequencies we want
        for i in range(0, len(dat.freqs)):
            if dat.freqs[i] >= freq_lower and dat.freqs[i] <= freq_upper:
                nfreqs.append(i)
        npdat = npdat[:, nfreqs, :]

        # get needed timept indices
        ntimes = list(range(350,701))
        npdat = npdat[:,:,ntimes]
        
        ppdat = np.mean(npdat, axis = 1) #calculate mean across frequencies
        ppdat = np.mean(ppdat, axis = 0) # calculate mean across channels
        
        allppdat.append(ppdat)
        
    if pcond == 0:
        beta_frontal_EarlyLateAligned.append(allppdat)
    elif pcond == 1:
        beta_frontal_EarlyRot.append(allppdat)
    elif pcond == 2:
        beta_frontal_LateRot.append(allppdat)
    elif pcond == 3:
        beta_frontal_EarlyMir.append(allppdat)
    elif pcond == 4:
        beta_frontal_LateMir.append(allppdat)
    elif pcond == 5:
        beta_frontal_EarlyRdm.append(allppdat)
    elif pcond == 6:
        beta_frontal_LateRdm.append(allppdat)
    

Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p000/frn\p000_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p001/frn\p001_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p002/frn\p002_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p003/frn\p003_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p004/frn\p004_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p005/frn\p005_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p006/frn\p006_early_late_aligned_power-tfr.h5 ...
App

Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p025/frn\p025_early_rot_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p026/frn\p026_early_rot_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p027/frn\p027_early_rot_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p028/frn\p028_early_rot_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p029/frn\p029_early_rot_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p030/frn\p030_early_rot_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p031/frn\p031_early_rot_power-tfr.h5 ...
Applying baseline co

Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p020/frn\p020_early_mir_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p021/frn\p021_early_mir_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p022/frn\p022_early_mir_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p023/frn\p023_early_mir_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p024/frn\p024_early_mir_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p025/frn\p025_early_mir_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p026/frn\p026_early_mir_power-tfr.h5 ...
Applying baseline co

Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p015/frn\p015_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p016/frn\p016_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p017/frn\p017_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p018/frn\p018_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p019/frn\p019_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p020/frn\p020_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p021/frn\p021_early_rdm_power-tfr.h5 ...
Applying baseline co

Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p009/frn\p009_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p010/frn\p010_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p011/frn\p011_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p012/frn\p012_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p013/frn\p013_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p014/frn\p014_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p015/frn\p015_early_late_aligned_power-tfr.h5 ...
App

Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p003/frn\p003_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p004/frn\p004_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p005/frn\p005_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p006/frn\p006_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p007/frn\p007_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p008/frn\p008_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p009/frn\p009_late_rot_power-tfr.h5 ...
Applying baseline correctio

Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p030/frn\p030_early_mir_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p031/frn\p031_early_mir_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p000/frn\p000_late_mir_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p001/frn\p001_late_mir_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p002/frn\p002_late_mir_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p003/frn\p003_late_mir_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p004/frn\p004_late_mir_power-tfr.h5 ...
Applying baseline correct

Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p025/frn\p025_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p026/frn\p026_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p027/frn\p027_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p028/frn\p028_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p029/frn\p029_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p030/frn\p030_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p031/frn\p031_early_rdm_power-tfr.h5 ...
Applying baseline co

In [9]:
# Get timepts to use for indices given by cluster-based permutation later and for timepts in saved data frames
# But only grab -0.25 to 1.5 sec time-locked to feedback onset (idx= 350:701)
root_directory = root
pp = 0 #only need one participant

# we can use aligned data
dat = load_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, task = 'early_late_aligned', output = "power")
time = dat[0].times
time = time[350:701] #get only timepoints we want

Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p000/frn\p000_early_late_aligned_power-tfr.h5 ...


In [10]:
# save tfr data across participants in each condition (used for plotting signals)
alpha_flists = [alpha_frontal_EarlyLateAligned, alpha_frontal_EarlyRot, alpha_frontal_LateRot, 
                alpha_frontal_EarlyRdm, alpha_frontal_LateRdm, 
                alpha_frontal_EarlyMir, alpha_frontal_LateMir]
beta_flists = [beta_frontal_EarlyLateAligned, beta_frontal_EarlyRot, beta_frontal_LateRot, 
               beta_frontal_EarlyRdm, beta_frontal_LateRdm,
               beta_frontal_EarlyMir, beta_frontal_LateMir]

alpha_conditionnames = ['alpha_earlylate_aligned', 'alpha_earlyrot', 'alpha_laterot', 'alpha_earlyrdm', 'alpha_laterdm', 'alpha_earlymir', 'alpha_latemir']
beta_conditionnames = ['beta_earlylate_aligned', 'beta_earlyrot', 'beta_laterot', 'beta_earlyrdm', 'beta_laterdm', 'beta_earlymir', 'beta_latemir']

freq_bands = ['alpha', 'beta']

for band in range(0, len(freq_bands)):
    if band == 0:
        for f in range(0, len(alpha_flists)):
            df = pd.DataFrame(alpha_flists[f][0]).T
            df['time'] = time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_frontal_%s_%s.csv' % (alpha_conditionnames[f], erps))
            df.to_csv(filename)
    elif band == 1:
        for f in range(0, len(beta_flists)):
            df = pd.DataFrame(beta_flists[f][0]).T
            df['time'] = time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_frontal_%s_%s.csv' % (beta_conditionnames[f], erps))
            df.to_csv(filename)
            

In [11]:
# First, we can compare each condition to aligned
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values
# This information can then be included in plots

alpha_flists = [alpha_frontal_EarlyRot, alpha_frontal_LateRot, 
                alpha_frontal_EarlyRdm, alpha_frontal_LateRdm, 
                alpha_frontal_EarlyMir, alpha_frontal_LateMir]
beta_flists = [beta_frontal_EarlyRot, beta_frontal_LateRot, 
               beta_frontal_EarlyRdm, beta_frontal_LateRdm,
               beta_frontal_EarlyMir, beta_frontal_LateMir]

alpha_conditionnames = ['alpha_earlyrot', 'alpha_laterot', 'alpha_earlyrdm', 'alpha_laterdm', 'alpha_earlymir', 'alpha_latemir']
beta_conditionnames = ['beta_earlyrot', 'beta_laterot', 'beta_earlyrdm', 'beta_laterdm', 'beta_earlymir', 'beta_latemir']

freq_bands = ['alpha', 'beta']
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

for band in range(0, len(freq_bands)):
    if band == 0:
        for f in range(0, len(alpha_flists)):
            T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_flists[f][0], alpha_frontal_EarlyLateAligned[0], p, perms)
#             print(clust_idx, clust_pvals)
            if len(clust_idx) == 0:
                condition.append(alpha_conditionnames[f])
                clust_idx_start.append(np.nan)
                clust_idx_end.append(np.nan)
                time_start.append(np.nan)
                time_end.append(np.nan)
                p_values.append(np.nan)
            else:
                for clust in range(0, len(clust_idx)):
                    cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                    cluster_start = cluster.start
                    clust_idx_start.append(cluster_start)
        
                    cluster_end = cluster.stop
                    clust_idx_end.append(cluster_end)
        
                    time_idx_start = time[cluster_start]
                    time_start.append(time_idx_start)
        
                    time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                    time_end.append(time_idx_end)
        
                    clust_p = clust_pvals[clust]
                    p_values.append(clust_p)
        
                    condition.append(alpha_conditionnames[f])
            
    elif band == 1:
        for f in range(0, len(beta_flists)):
            T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_flists[f][0], beta_frontal_EarlyLateAligned[0], p, perms)
#             print(clust_idx, clust_pvals)
            if len(clust_idx) == 0:
                condition.append(beta_conditionnames[f])
                clust_idx_start.append(np.nan)
                clust_idx_end.append(np.nan)
                time_start.append(np.nan)
                time_end.append(np.nan)
                p_values.append(np.nan)
            else:
                for clust in range(0, len(clust_idx)):
                    cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                    cluster_start = cluster.start
                    clust_idx_start.append(cluster_start)
        
                    cluster_end = cluster.stop
                    clust_idx_end.append(cluster_end)
        
                    time_idx_start = time[cluster_start]
                    time_start.append(time_idx_start)
        
                    time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                    time_end.append(time_idx_end)
        
                    clust_p = clust_pvals[clust]
                    p_values.append(clust_p)
        
                    condition.append(beta_conditionnames[f])
        
perm_test = pd.DataFrame(
    {'condition': condition,
        'clust_idx_start': clust_idx_start,
        'clust_idx_end': clust_idx_end,
        'time_start': time_start,
        'time_end': time_end,
        'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_Permutation_test_vsAligned_%s.csv' % (erps))
perm_test.to_csv(perm_test_filename)


stat_fun(H1): min=-2.904708 max=-0.512092
Running initial clustering …
Found 5 clusters


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-2.145043 max=0.942834
Running initial clustering …
Found 1 cluster


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-3.993595 max=-0.793125
Running initial clustering …
Found 2 clusters


...

  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-2.566809 max=0.110977
Running initial clustering …
Found 2 clusters


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-3.072000 max=-0.037843
Running initial clustering …
Found 3 clusters


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

In [12]:
# Next, we subtract aligned from each condition, so that we can compare early vs late in each perturbation type

freq_bands = ['alpha', 'beta']
diffconds = ['earlyrot', 'laterot', 'earlyrdm', 'laterdm', 'earlymir', 'latemir']

alpha_frontal_earlyrot_diff = []
alpha_frontal_laterot_diff = []
alpha_frontal_earlyrdm_diff = []
alpha_frontal_laterdm_diff = []
alpha_frontal_earlymir_diff = []
alpha_frontal_latemir_diff = []

beta_frontal_earlyrot_diff = []
beta_frontal_laterot_diff = []
beta_frontal_earlyrdm_diff = []
beta_frontal_laterdm_diff = []
beta_frontal_earlymir_diff = []
beta_frontal_latemir_diff = []

for band in range(0, len(freq_bands)):
    if band == 0:
        for cond in range(0, len(diffconds)):
            if cond == 0:
                diffevks = np.subtract(alpha_frontal_EarlyRot[0], alpha_frontal_EarlyLateAligned[0])
                alpha_frontal_earlyrot_diff.append(diffevks)
                alpha_frontal_earlyrot_diff = alpha_frontal_earlyrot_diff[0] #to keep shape of object consistent
            elif cond == 1:
                diffevks = np.subtract(alpha_frontal_LateRot[0], alpha_frontal_EarlyLateAligned[0])
                alpha_frontal_laterot_diff.append(diffevks)
                alpha_frontal_laterot_diff = alpha_frontal_laterot_diff[0]
            elif cond == 2:
                diffevks = np.subtract(alpha_frontal_EarlyRdm[0], alpha_frontal_EarlyLateAligned[0])
                alpha_frontal_earlyrdm_diff.append(diffevks)
                alpha_frontal_earlyrdm_diff = alpha_frontal_earlyrdm_diff[0]
            elif cond == 3:
                diffevks = np.subtract(alpha_frontal_LateRdm[0], alpha_frontal_EarlyLateAligned[0])
                alpha_frontal_laterdm_diff.append(diffevks)
                alpha_frontal_laterdm_diff = alpha_frontal_laterdm_diff[0]
            elif cond == 4:
                diffevks = np.subtract(alpha_frontal_EarlyMir[0], alpha_frontal_EarlyLateAligned[0])
                alpha_frontal_earlymir_diff.append(diffevks)
                alpha_frontal_earlymir_diff = alpha_frontal_earlymir_diff[0]
            elif cond == 5:
                diffevks = np.subtract(alpha_frontal_LateMir[0], alpha_frontal_EarlyLateAligned[0])
                alpha_frontal_latemir_diff.append(diffevks)
                alpha_frontal_latemir_diff = alpha_frontal_latemir_diff[0]
    elif band == 1:
        for cond in range(0, len(diffconds)):
            if cond == 0:
                diffevks = np.subtract(beta_frontal_EarlyRot[0], beta_frontal_EarlyLateAligned[0])
                beta_frontal_earlyrot_diff.append(diffevks)
                beta_frontal_earlyrot_diff = beta_frontal_earlyrot_diff[0] #to keep shape of object consistent
            elif cond == 1:
                diffevks = np.subtract(beta_frontal_LateRot[0], beta_frontal_EarlyLateAligned[0])
                beta_frontal_laterot_diff.append(diffevks)
                beta_frontal_laterot_diff = beta_frontal_laterot_diff[0]
            elif cond == 2:
                diffevks = np.subtract(beta_frontal_EarlyRdm[0], beta_frontal_EarlyLateAligned[0])
                beta_frontal_earlyrdm_diff.append(diffevks)
                beta_frontal_earlyrdm_diff = beta_frontal_earlyrdm_diff[0]
            elif cond == 3:
                diffevks = np.subtract(beta_frontal_LateRdm[0], beta_frontal_EarlyLateAligned[0])
                beta_frontal_laterdm_diff.append(diffevks)
                beta_frontal_laterdm_diff = beta_frontal_laterdm_diff[0]
            elif cond == 4:
                diffevks = np.subtract(beta_frontal_EarlyMir[0], beta_frontal_EarlyLateAligned[0])
                beta_frontal_earlymir_diff.append(diffevks)
                beta_frontal_earlymir_diff = beta_frontal_earlymir_diff[0]
            elif cond == 5:
                diffevks = np.subtract(beta_frontal_LateMir[0], beta_frontal_EarlyLateAligned[0])
                beta_frontal_latemir_diff.append(diffevks)
                beta_frontal_latemir_diff = beta_frontal_latemir_diff[0]

In [13]:
# save aligned (baseline) subtracted tfr data across participants in each condition (used for plotting signals)
alpha_flists = [alpha_frontal_earlyrot_diff, alpha_frontal_laterot_diff, 
                alpha_frontal_earlyrdm_diff, alpha_frontal_laterdm_diff, 
                alpha_frontal_earlymir_diff, alpha_frontal_latemir_diff]
beta_flists = [beta_frontal_earlyrot_diff, beta_frontal_laterot_diff, 
               beta_frontal_earlyrdm_diff, beta_frontal_laterdm_diff, 
               beta_frontal_earlymir_diff, beta_frontal_latemir_diff]

alpha_conditionnames = ['alpha_earlyrot', 'alpha_laterot', 'alpha_earlyrdm', 'alpha_laterdm', 'alpha_earlymir', 'alpha_latemir']
beta_conditionnames = ['beta_earlyrot', 'beta_laterot', 'beta_earlyrdm', 'beta_laterdm', 'beta_earlymir', 'beta_latemir']

freq_bands = ['alpha', 'beta']

for band in range(0, len(freq_bands)):
    if band == 0:
        for f in range(0, len(alpha_flists)):
            df = pd.DataFrame(alpha_flists[f]).T #no need to index [0]
            df['time'] = time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_frontal_DiffWaves_%s_%s.csv' % (alpha_conditionnames[f], erps))
            df.to_csv(filename)
    elif band == 1:
        for f in range(0, len(beta_flists)):
            df = pd.DataFrame(beta_flists[f]).T
            df['time'] = time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_frontal_DiffWaves_%s_%s.csv' % (beta_conditionnames[f], erps))
            df.to_csv(filename)

In [14]:
# Compare EARLY VS LATE for each perturbation
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values
# This information can then be included in plots
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

freq_bands = ['alpha', 'beta']

for band in range(0, len(freq_bands)):
    if band == 0:
        alpha_conditionnames = ['alpha_frontal_rot', 'alpha_frontal_rdm', 'alpha_frontal_mir']
        for c in range(0, len(alpha_conditionnames)):
            if c == 0:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_frontal_laterot_diff, alpha_frontal_earlyrot_diff, p, perms)
        #         print(clust_idx, clust_pvals)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
            elif c == 1:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_frontal_laterdm_diff, alpha_frontal_earlyrdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
            elif c == 2:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_frontal_latemir_diff, alpha_frontal_earlymir_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
    elif band == 1:
        beta_conditionnames = ['beta_frontal_rot', 'beta_frontal_rdm', 'beta_frontal_mir']
        for c in range(0, len(beta_conditionnames)):
            if c == 0:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_frontal_laterot_diff, beta_frontal_earlyrot_diff, p, perms)
        #         print(clust_idx, clust_pvals)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
            
            elif c == 1:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_frontal_laterdm_diff, beta_frontal_earlyrdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
            
            elif c == 2:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_frontal_latemir_diff, beta_frontal_earlymir_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
        
perm_test = pd.DataFrame(
    {'condition': condition,
     'clust_idx_start': clust_idx_start,
     'clust_idx_end': clust_idx_end,
     'time_start': time_start,
     'time_end': time_end,
     'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_Permutation_test_EarlyvsLate_%s.csv' % (erps))
perm_test.to_csv(perm_test_filename)

stat_fun(H1): min=-0.175199 max=2.903530
Running initial clustering …
Found 1 cluster


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-0.408850 max=1.799028
Running initial clustering …
Found 0 clusters
stat_fun(H1): min=-1.635287 max=0.946400
Running initial clustering …
Found 0 clusters
stat_fun(H1): min=-0.315408 max=1.134542
Running initial clustering …
Found 0 clusters
stat_fun(H1): min=-0.411780 max=2.069011
Running initial clustering …
Found 1 cluster


C:\Users\Raphael\AppData\Local\Temp\ipykernel_22092\3082407755.py:11: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh,
C:\Users\Raphael\AppData\Local\Temp\ipykernel_22092\3082407755.py:11: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh,
C:\Users\Raphael\AppData\Local\Temp\ipykernel_22092\3082407755.py:11: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh,


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-1.212308 max=1.570032
Running initial clustering …
Found 0 clusters


C:\Users\Raphael\AppData\Local\Temp\ipykernel_22092\3082407755.py:11: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh,


In [17]:
# Next step is to subtract early from late condition, to generate a single signal for each perturbation
diffconds = ['rot', 'rdm', 'mir']
freq_bands = ['alpha', 'beta']

alpha_frontal_rot_diff = []
alpha_frontal_rdm_diff = []
alpha_frontal_mir_diff = []

beta_frontal_rot_diff = []
beta_frontal_rdm_diff = []
beta_frontal_mir_diff = []

for band in range(0, len(freq_bands)):
    if band == 0:
        for cond in range(0, len(diffconds)):
            if cond == 0:
                diffevks = np.subtract(alpha_frontal_laterot_diff, alpha_frontal_earlyrot_diff)
                alpha_frontal_rot_diff.append(diffevks)
                alpha_frontal_rot_diff = alpha_frontal_rot_diff[0] #to keep shape of object consistent
            elif cond == 1:
                diffevks = np.subtract(alpha_frontal_laterdm_diff, alpha_frontal_earlyrdm_diff)
                alpha_frontal_rdm_diff.append(diffevks)
                alpha_frontal_rdm_diff = alpha_frontal_rdm_diff[0]
            elif cond == 2:
                diffevks = np.subtract(alpha_frontal_latemir_diff, alpha_frontal_earlymir_diff)
                alpha_frontal_mir_diff.append(diffevks)
                alpha_frontal_mir_diff = alpha_frontal_mir_diff[0]
    elif band == 1:
        for cond in range(0, len(diffconds)):
            if cond == 0:
                diffevks = np.subtract(beta_frontal_laterot_diff, beta_frontal_earlyrot_diff)
                beta_frontal_rot_diff.append(diffevks)
                beta_frontal_rot_diff = beta_frontal_rot_diff[0] #to keep shape of object consistent
            elif cond == 1:
                diffevks = np.subtract(beta_frontal_laterdm_diff, beta_frontal_earlyrdm_diff)
                beta_frontal_rdm_diff.append(diffevks)
                beta_frontal_rdm_diff = beta_frontal_rdm_diff[0]
            elif cond == 2:
                diffevks = np.subtract(beta_frontal_latemir_diff, beta_frontal_earlymir_diff)
                beta_frontal_mir_diff.append(diffevks)
                beta_frontal_mir_diff = beta_frontal_mir_diff[0]

In [18]:
# save early and late subtracted tfr data across participants in each condition (used for plotting signals)
alpha_flists = [alpha_frontal_rot_diff, alpha_frontal_rdm_diff, alpha_frontal_mir_diff]
beta_flists = [beta_frontal_rot_diff, beta_frontal_rdm_diff, beta_frontal_mir_diff]

alpha_conditionnames = ['alpha_rot', 'alpha_rdm', 'alpha_mir']
beta_conditionnames = ['beta_rot', 'beta_rdm', 'beta_mir']

freq_bands = ['alpha', 'beta']

for band in range(0, len(freq_bands)):
    if band == 0:
        for f in range(0, len(alpha_flists)):
            df = pd.DataFrame(alpha_flists[f]).T #no need to index [0]
            df['time'] = time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_frontal_DiffWaves_EvL_%s_%s.csv' % (alpha_conditionnames[f], erps))
            df.to_csv(filename)
    elif band == 1:
        for f in range(0, len(beta_flists)):
            df = pd.DataFrame(beta_flists[f]).T
            df['time'] = time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_frontal_DiffWaves_EvL_%s_%s.csv' % (beta_conditionnames[f], erps))
            df.to_csv(filename)

In [19]:
# Compare EARLY VS LATE for each perturbation
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values
# This information can then be included in plots
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

freq_bands = ['alpha', 'beta']

for band in range(0, len(freq_bands)):
    if band == 0:
        alpha_conditionnames = ['alpha_frontal_rotvmir', 'alpha_frontal_rotvrdm', 'alpha_frontal_mirvrdm']
        for c in range(0, len(alpha_conditionnames)):
            if c == 0:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_frontal_rot_diff, alpha_frontal_mir_diff, p, perms)
        #         print(clust_idx, clust_pvals)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
            elif c == 1:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_frontal_rot_diff, alpha_frontal_rdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
            elif c == 2:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_frontal_mir_diff, alpha_frontal_rdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
    elif band == 1:
        beta_conditionnames = ['beta_frontal_rotvmir', 'beta_frontal_rotvrdm', 'beta_frontal_mirvrdm']
        for c in range(0, len(beta_conditionnames)):
            if c == 0:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_frontal_rot_diff, beta_frontal_mir_diff, p, perms)
        #         print(clust_idx, clust_pvals)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
            
            elif c == 1:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_frontal_rot_diff, beta_frontal_rdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
            
            elif c == 2:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_frontal_mir_diff, beta_frontal_rdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
        
perm_test = pd.DataFrame(
    {'condition': condition,
     'clust_idx_start': clust_idx_start,
     'clust_idx_end': clust_idx_end,
     'time_start': time_start,
     'time_end': time_end,
     'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_Permutation_test_PerturbTypeComp_%s.csv' % (erps))
perm_test.to_csv(perm_test_filename)

stat_fun(H1): min=-0.428329 max=2.274866
Running initial clustering …
Found 1 cluster


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-1.084491 max=1.074334
Running initial clustering …
Found 0 clusters
stat_fun(H1): min=-2.183883 max=0.692505
Running initial clustering …
Found 1 cluster


C:\Users\Raphael\AppData\Local\Temp\ipykernel_22092\3082407755.py:11: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh,


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-1.696605 max=1.483973
Running initial clustering …
Found 0 clusters
stat_fun(H1): min=-1.492816 max=0.304655
Running initial clustering …
Found 0 clusters
stat_fun(H1): min=-2.222081 max=1.170338
Running initial clustering …
Found 3 clusters


C:\Users\Raphael\AppData\Local\Temp\ipykernel_22092\3082407755.py:11: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh,
C:\Users\Raphael\AppData\Local\Temp\ipykernel_22092\3082407755.py:11: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh,


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]